## Let try to build a basic Ai agent ;; at intial it will only search the web if it feels like it has to search something on web

In [1]:
import os
os.environ["OPENAI_API_KEY"]= ""


In [23]:
from IPython.display import display, HTML

display(HTML("<style>pre {white-space: pre-wrap;}</style>"))


In [2]:
import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning)


In [5]:
!pip uninstall -y langchain langchain-core langchain-community langchain-openai


In [6]:
!pip install -U \
  langchain==1.2.0 \
  langchain-core>=1.2.1 \
  langchain-community>=1.2.1 \
  langchain-openai>=0.2.0 \
  duckduckgo-search

In [7]:
!pip install -U ddgs

In [8]:
from langchain_openai import ChatOpenAI
from langchain_core.tools import tool
import requests

In [25]:
from langchain_community.tools import DuckDuckGoSearchRun

search_tool= DuckDuckGoSearchRun()

results= search_tool.invoke("top news in india today")
results

'Top - India - news - today ... info@truescoopnews.com ... Copyright © 2025 TrueScoop News . ... News 24 is your stop for those who are inquisitive enough to keep abreast with the ... Today News Headlines in India : Stay Updated with DLSNews24 ... News 24 is your stop for those who are inquisitive enough to keep abreast with the ... Today News Headlines in India : Stay Updated with DLSNews24 Home News Today ’s Trending News : Top Headlines in India and ... Today ’s Trending News : Top Headlines in India and World Updates (August 25, 2025) For readers who want a quick but comprehensive roundup, here are the top 10 today ’s news updates in India . ... trending topics in today ’s news ...'

In [10]:
llm= ChatOpenAI()

## now main code is start

In [12]:
!pip install -U langchainhub

In [15]:
from langchain.agents import create_agent


In [16]:
import langchain
import langchain_core
import langchain_community
import langchain_openai

print(langchain.__version__)
print(langchain_core.__version__)
print(langchain_community.__version__)
# print(langchain_openai.__version__)


1.2.0
1.2.5
0.4.1


## if you want pre-define prompt you can use langchain import hub
## so you have to pull ReAct prompt from langchain hub ;; pull the standard ReAct prompt;; bz we are going to build a react agent;; ReAct means (Reasoning + Action)

In [20]:
from langchain.agents.middleware import wrap_tool_call
from langchain.messages import ToolMessage, HumanMessage

@wrap_tool_call
def handle_tool_errors(request, handler):
  """ Handle tool execution error with custom message """
  try:
    return handler(request)
  except Exception as e:
    # return a custom toolmessage to the model
    return ToolMessage(
        content=f"Tool error: Please check input and try again. ({str(e)})",
        tool_call_id=request.tool_call['id']
    )


In [21]:
# create  ReAct agent manually with the react prompt

agent= create_agent(
    model=llm,
    tools=[search_tool],
    middleware=[handle_tool_errors]
)

results= agent.invoke({"messages":[HumanMessage("Identify the current most popular wireless headphones and verify availability.")]})
results

{'messages': [HumanMessage(content='Identify the current most popular wireless headphones and verify availability.', additional_kwargs={}, response_metadata={}, id='e6244e08-ba8e-4526-846a-2c0ef7e591bf'), AIMessage(content='', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 21, 'prompt_tokens': 88, 'total_tokens': 109, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_provider': 'openai', 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'id': 'chatcmpl-CsVdB4Dd2cJAolLdJY6LsZ61YbCid', 'service_tier': 'default', 'finish_reason': 'tool_calls', 'logprobs': None}, id='lc_run--019b6fce-1629-7450-9f7b-45a11fc3dc80-0', tool_calls=[{'name': 'duckduckgo_search', 'args': {'query': 'current most popular wireless headphones'}, 'id': 'call_bMQRohgJ62uY09xPuVoAGVNK', 'type': 'tool_